In [262]:
from os import listdir, PathLike
from os.path import join
from typing import List, Tuple, Dict
from dataclasses import dataclass
import h5py
import numpy as np
import pandas as pd
from hdmf.backends.hdf5 import H5DataIO
from pynwb import NWBFile, TimeSeries
from pynwb.file import Subject
from pynwb.ecephys import ElectricalSeries, ElectrodeGroup, LFP
from pynwb.behavior import BehavioralEvents
import nixio
import regex as re
from usz_neuro_conversion.common import (
    SessionContext,
    NixContext,
    get_metadata_row,
    read_nix,
    get_date,
    write_nwb,
    standardize_sex,
    find_nix_files, get_matlab_matrix_scalars_ragged, get_micro_dir
)
from joblib import Parallel, delayed
import multiprocessing
import nixio
from usz_neuro_conversion.common import read_nix, NixContext

# Verbal Task

In [263]:
def read_matlab(ctx: SessionContext):
    global micros
    if len(micros) > 0:
        micros = {}
    micro_files = _find_micro_data_files(ctx)
    files = micro_files[ctx.subject]

    def read_electrode(electrode, file):
        with h5py.File(file, 'r') as file:
            trials = get_matlab_matrix(file, "trial")
            times = get_matlab_matrix(file, "time")
            return {
                electrode: Micro(
                    trials=trials,
                    times=times
                )
            }

    num_cores = multiprocessing.cpu_count()
    # Source: https://stackoverflow.com/a/50926231
    micros = Parallel(n_jobs=num_cores)(delayed(read_electrode)(electrode, file) for electrode, file in files.items())
    # Source: https://stackoverflow.com/a/43219379
    micros = {k: v for d in micros for k, v in d.items()}
    assert len(micros) > 0

In [3]:
def get_matlab_matrix(file: h5py.File, variable: str) -> np.ndarray:
    ref = (
        file.get(f"data/{variable}")
        if "data" in file.keys()
        else file.get(f"dataMicro/{variable}")
    )
    refs = [ref[0] for ref in ref]
    assert len(refs) > 0

    inner_dim = np.array(file[ref[0][0]][:]).shape
    matrices = np.zeros((len(refs), inner_dim[0], inner_dim[1]))
    for i, ref in enumerate(refs):
        matrices[i] = file[ref][:]
    return matrices

In [4]:
@dataclass(frozen=True)
class Micro:
    trials: np.ndarray
    times: np.ndarray


micros = {}

In [5]:
def _find_micro_data_files(ctx: SessionContext) -> Dict[int, Dict[str, PathLike]]:
    dir = get_micro_dir(ctx)
    micro_files = {}
    for file in listdir(dir):
        match = MATLAB_RE.match(file)
        if match:
            subject, _electrode_index, electrode = match.groups()
            subject = int(subject)
            if subject not in CORRECTED_PATIENT:
                continue
            subject = CORRECTED_PATIENT[subject]
            if subject not in micro_files:
                micro_files[subject] = {}
            micro_files[subject][electrode] = join(dir, file)
    assert len(micro_files) > 0
    return micro_files


In [6]:
CORRECTED_PATIENT = {
    28: 1,
    22: 2,
    19: 3,
    30: 4,
    33: 5,
    13: 6,
    23: 7,
    29: 8,
    16: 9,
}

In [7]:
def create_context(subject: int, session: int) -> SessionContext:
    nix_context = NixContext(
        subject, session, project="Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM"
    )
    nix = read_nix(nix_context)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description="Running experiment as described in the the experiment description",
        identifier=f"Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject{subject:02}_session{session:02}",
        session_start_time=get_date(nix_context),
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        experimenter="Boran, Ece",
        keywords=[
            "Neuroscience",
            "Electrophysiology",
            "Human",
            "Awake",
            "Local field potential",
            "Neuronal action potential",
            "Spikes",
            "Medial temporal lobe",
            "Hippocampus",
            "Entorhinal cortex",
            "Amygdala",
            "Scalp EEG",
            "Intracranial EEG",
            "Cognitive task",
            "Verbal working memory",
            "Epilepsy",
        ],
    )
    return nix_context.to_session_context(nix, nwb)

In [8]:
# Micro_Data_Patient_04_Electrode_01_uAR
MATLAB_RE = re.compile(r"Micro_Data_Patient_(\d+)_Electrode_(\d+)_u([A-Z]+).mat")

In [18]:
subject = 1
session = 1
ctx = create_context(subject, session)
read_matlab(ctx)

KeyboardInterrupt: 

In [ ]:
micros.keys()

In [ ]:
print(micros["AHL"].trials.shape)
print(micros["AHL"].times.shape)

In [ ]:
trials = np.zeros(
    (len(micros), micros["AHL"].trials.shape[0], micros["AHL"].trials.shape[1], micros["AHL"].trials.shape[2]))
for i, (electrode, micro) in enumerate(micros.items()):
    trials[i] = micro.trials
trials = trials.swapaxes(2, 3).swapaxes(1, 2)
# electrode - channel/subelectrode - trial - values per timestamp
trials.shape

In [ ]:
trials_reshaped = np.zeros((trials.shape[0] * trials.shape[1], trials.shape[2], trials.shape[3]))
for i in range(trials.shape[0]):
    for j in range(trials.shape[1]):
        trials_reshaped[i * trials.shape[1] + j] = trials[i, j]
trials_reshaped.shape

In [ ]:
trials_reshaped_again = np.zeros((trials_reshaped.shape[0], trials_reshaped.shape[1] * trials_reshaped.shape[2]))
for i in range(trials_reshaped.shape[1]):
    for j in range(trials_reshaped.shape[2]):
        trials_reshaped_again[:, i * trials_reshaped.shape[1] + j] = trials_reshaped[:, i, j]
trials_reshaped_again = trials_reshaped_again.transpose()
trials_reshaped_again.shape

In [ ]:
times = np.zeros(micros["AHL"].times.shape[0] * micros["AHL"].times.shape[1])
for i in range(micros["AHL"].times.shape[0]):
    for j in range(micros["AHL"].times.shape[1]):
        times[i * micros["AHL"].times.shape[1] + j] = micros["AHL"].times[i, j, 0]
times.shape

# New Approach

In [9]:
info = None


def get_matlab_trial_info(ctx: SessionContext) -> pd.DataFrame:
    global info
    if info is not None:
        return info
    micro_files = _find_micro_data_files(ctx)
    files = micro_files[ctx.subject]
    reference_file = list(files.values())[0]  # arbitrary
    csv_name = reference_file.replace(".mat", ".csv")
    with open(csv_name, "r") as file:
        info = pd.read_csv(file, sep=",")
    return info


def get_trial_indices(ctx: SessionContext) -> List[int]:
    micro_info = get_matlab_trial_info(ctx)
    # Source: https://stackoverflow.com/a/17215844
    x = micro_info.loc[:, "Session"] == ctx.session
    return x[x].index.values


@dataclass(frozen=True)
class MicroData:
    matrix: np.ndarray
    measurements_per_trial: int
    channels_per_electrode: int
    electrodes: int
    trial_info: pd.DataFrame
    timestamps_per_trial: np.ndarray


def prepare_micro_data(ctx: SessionContext) -> MicroData:
    micro_files = _find_micro_data_files(ctx)
    files = micro_files[ctx.subject]

    num_electrodes = len(files)
    reference_file = list(files.values())[0]  # arbitrary

    with h5py.File(reference_file, 'r') as file:
        ref = file.get("dataMicro/trial")
        refs = [ref[0] for ref in ref]

        inner_dim = np.array(file[refs[0]][:]).shape
        measurements = inner_dim[0]
        channels = inner_dim[1]

        trial_info = get_matlab_trial_info(ctx)
        trials_in_current_session_indices = get_trial_indices(ctx)

        total_sources = num_electrodes * channels
        total_measurements = measurements * len(trials_in_current_session_indices)
        matrix = np.zeros((total_measurements, total_sources))

        times_ref = file.get("dataMicro/time")[0][0]
        times = file[times_ref][:][:, 0]

        return MicroData(
            matrix=matrix,
            measurements_per_trial=measurements,
            channels_per_electrode=channels,
            electrodes=num_electrodes,
            trial_info=trial_info,
            timestamps_per_trial=times
        )


def read_lfp_trials(ctx: SessionContext, micro_data: MicroData):
    micro_files = _find_micro_data_files(ctx)
    files = micro_files[ctx.subject]
    indices_in_session = get_trial_indices(ctx)
    for electrode_index, file in enumerate(files.values()):
        with h5py.File(file, 'r') as electrode_file:
            refs = [ref[0] for ref in electrode_file.get(f"dataMicro/trial")]
            trial_refs = [refs[i] for i in indices_in_session]
            for trial_index, trial_ref in enumerate(trial_refs):
                measurement_index = trial_index * micro_data.measurements_per_trial
                channel_index = electrode_index * micro_data.channels_per_electrode
                data = electrode_file[trial_ref]
                micro_data.matrix[measurement_index:measurement_index + micro_data.measurements_per_trial,
                channel_index:channel_index + micro_data.channels_per_electrode] = data

In [45]:
subject = 6
session = 1
ctx = create_context(subject, session)

micro_data = prepare_micro_data(ctx)
print("matrix.shape", micro_data.matrix.shape)
print("measurements_per_trial", micro_data.measurements_per_trial)
print("channels_per_electrode", micro_data.channels_per_electrode)
print("electrodes", micro_data.electrodes)
print("trial_info.shape", micro_data.trial_info.shape)

matrix.shape (12800000, 64)
measurements_per_trial 256000
channels_per_electrode 8
electrodes 8
trial_info.shape (349, 15)


In [11]:
read_lfp_trials(ctx, micro_data)

In [12]:
print(micro_data.matrix[9, 4])

-5.2339244549504125


In [13]:
print(micro_data.matrix[0, 0])


3.3570360935542003


In [14]:
def _get_total_time_before(ctx: SessionContext, trial: int):
    info = get_matlab_trial_info(ctx)
    start_time = \
        info.loc[(info.Session == ctx.session) & (info.nTrial_Full == 1), "TimeStartS"].values[
            0]
    end_time = info.loc[(info.Session == ctx.session) & (info.nTrial_Full == trial + 1), "TimeStartS"].values[0]
    return end_time - start_time


def _get_total_time_after(ctx: SessionContext, trial: int):
    info = get_matlab_trial_info(ctx)
    start_time = \
        info.loc[(info.Session == ctx.session) & (info.nTrial_Full == 1), "TimeStartS"].values[
            0]
    end_time = info.loc[(info.Session == ctx.session) & (info.nTrial_Full == trial + 1), "TimeStopS"].values[0]
    return end_time - start_time

In [15]:
_get_total_time_before(ctx, trial=1)

12.58437000000049

In [16]:
def read_lfp_timestamps(ctx: SessionContext, micro_data: MicroData) -> np.ndarray:
    measurements = micro_data.measurements_per_trial
    assert micro_data.timestamps_per_trial.shape[0] == measurements

    total_measurements = micro_data.matrix.shape[0]
    timestamps = np.zeros(total_measurements)
    trials = total_measurements // measurements
    assert total_measurements % measurements == 0
    assert trials > 0
    for trial in range(trials):
        offset = -6.0
        trial_timestamps = micro_data.timestamps_per_trial - offset + _get_total_time_before(ctx, trial)
        index = trial * measurements
        timestamps[index:index + measurements] = trial_timestamps
    return timestamps


In [17]:
timestamps = read_lfp_timestamps(ctx, micro_data)

In [18]:
print(timestamps[-1])

548.1943147500002


In [49]:
def _get_ieeg_electrode_labels_including_lfp(ctx: SessionContext) -> List[str]:
    micro_files = _find_micro_data_files(ctx)[ctx.subject]
    labels = []
    for file in micro_files.values():
        with h5py.File(file, 'r') as file:
            for refs in file.get("dataMicro/label"):
                for ref in refs:
                    labels.append(file[ref][:])
    labels = [[char[0] for char in label] for label in labels]
    # int to utf-8
    labels = ["".join(map(chr, label)) for label in labels]
    # trim leading 'u'
    labels = [label[1:] for label in labels]
    return labels

In [50]:
labels = _get_ieeg_electrode_labels_including_lfp(ctx)

In [48]:
print(labels)

['AHL1', 'AHL2', 'AHL3', 'AHL4', 'AHL5', 'AHL6', 'AHL7', 'AHL8', 'AHR1', 'AHR2', 'AHR3', 'AHR4', 'AHR5', 'AHR6', 'AHR7', 'AHR8', 'AL1', 'AL2', 'AL3', 'AL4', 'AL5', 'AL6', 'AL7', 'AL8', 'AR1', 'AR2', 'AR3', 'AR4', 'AR5', 'AR6', 'AR7', 'AR8', 'ECL1', 'ECL2', 'ECL3', 'ECL4', 'ECL5', 'ECL6', 'ECL7', 'ECL8', 'ECR1', 'ECR2', 'ECR3', 'ECR4', 'ECR5', 'ECR6', 'ECR7', 'ECR8', 'PHL1', 'PHL2', 'PHL3', 'PHL4', 'PHL5', 'PHL6', 'PHL7', 'PHL8', 'PHR1', 'PHR2', 'PHR3', 'PHR4', 'PHR5', 'PHR6', 'PHR7', 'PHR8']


In [141]:
print(len(labels))

64


# Visual Task

In [268]:
def _find_micro_data_files(ctx: SessionContext) -> Dict[int, Dict[int, Dict[int, PathLike]]]:
    dir = get_micro_dir(ctx)
    micro_files = {}
    for file in listdir(dir):
        match = MATLAB_RE.match(file)
        if match:
            subject, session, part = map(int, match.groups())
            if subject not in CORRECTED_PATIENT:
                continue
            subject = CORRECTED_PATIENT[subject]
            if subject not in micro_files:
                micro_files[subject] = {}
            if session not in micro_files[subject]:
                micro_files[subject][session] = {}
            micro_files[subject][session][part] = join(dir, file)
    return micro_files

In [269]:
# Created manually because the provided list was wrong...
CORRECTED_PATIENT = {
    13: 1,
    34: 2,
    29: 3,
    30: 4,
    37: 5,
    35: 6,
    38: 7,
    40: 8,
    19: 9,
    23: 10,
    41: 11,
    28: 12,
    22: 13,
}

In [270]:
# Micro_Intervals_Patient_19_Session_01_Part_01_Interval_0_NaN_s
MATLAB_RE = re.compile(r"Micro_Intervals_Patient_(\d+)_Session_(\d+)_Part_(\d+)_Interval_0_NaN_s.mat")

In [271]:
def create_context(subject: int, session: int) -> SessionContext:
    nix_context = NixContext(subject, session, project="Human_MTL_units_visual_WM")
    nix = read_nix(nix_context)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description="Running experiment as described in the the experiment description",
        identifier=f"Human_MTL_units_visual_WM_subject{subject:02}_session{session:02}",
        session_start_time=get_date(nix_context),
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        experimenter="Boran, Ece",
        keywords=[
            "Visual",
            "Spatial",
            "Neural decoding",
            "Hippocampus",
            "Entorhinal cortex",
        ],
    )
    return nix_context.to_session_context(nix, nwb)

In [272]:
def read_lfp_trials(ctx: SessionContext):
    micro_files = _find_micro_data_files(ctx)
    files = micro_files[ctx.subject][ctx.session]
    for part, file in files.items():
        with h5py.File(file, 'r') as file:
            ref = file.get("dataMicro/trial")
            refs = [ref[0] for ref in ref]
            for ref in refs:
                data = file[ref][:]
                print(data.shape)

In [273]:
from usz_neuro_conversion.common import _get_in_dir


def find_dirty_csvs(project: str) -> List[str]:
    csvdir = join(_get_in_dir(), "to_convert", project, "micro_data", "dirty")
    return [join(csvdir, file) for file in listdir(csvdir)]


In [294]:
project = "Human_MTL_units_visual_WM"
micro_files = _find_micro_data_files(ctx)
infos = []
for file_name in find_dirty_csvs(project):
    with open(file_name, "r") as file:
        info = pd.read_csv(file, sep=",")
        infos.append(info)
        patient = str(info.Patient.values[0])
        substring = f"Patient_{patient}_Session_"
        assert substring in file_name



In [275]:
# remove rows where ResponseTime is NaN
infos = [info[~info.ResponseTime.isna()] for info in infos]

In [276]:

response_times = {}
for subject, sessions in find_nix_files(project).items():
    for session, _ in sessions.items():
        context = create_context(subject, session)
        times = [trial.props[-1].values[0] for trial in
                 context.nix.sections["Session"].sections["Trial properties"].sections]
        if context.subject not in response_times:
            response_times[context.subject] = {}
        response_times[context.subject][context.session] = times
print(len(response_times))

13


In [277]:
mapped_infos = {}
for subject, sessions in response_times.items():
    for session, times in sessions.items():
        needle = times[0]
        right_info = None
        for i, info in enumerate(infos):
            candidate = info.ResponseTime.values[0]
            if np.isclose(candidate, needle):
                right_info = i
                break
        if right_info is None:
            print(f"Could not find info for subject {subject} session {session}")
        else:
            info = infos[right_info]
            patient = info.Patient.values[0]
            part = info.Part.values[0]
            print(f"Found info for subject {subject} session {session}: patient {patient} part {part}")
            if subject not in mapped_infos:
                mapped_infos[subject] = {}
            mapped_infos[subject][session] = info

Found info for subject 1 session 1: patient 13 part 1
Found info for subject 2 session 1: patient 34 part 1
Found info for subject 3 session 1: patient 29 part 1
Found info for subject 3 session 2: patient 29 part 1
Found info for subject 4 session 1: patient 30 part 1
Found info for subject 5 session 1: patient 37 part 1
Found info for subject 6 session 1: patient 35 part 1
Found info for subject 7 session 1: patient 38 part 1
Found info for subject 8 session 1: patient 40 part 1
Found info for subject 8 session 2: patient 40 part 1
Found info for subject 9 session 1: patient 19 part 1
Found info for subject 10 session 1: patient 23 part 1
Found info for subject 11 session 1: patient 41 part 1
Found info for subject 12 session 1: patient 28 part 1
Found info for subject 12 session 2: patient 28 part 1
Found info for subject 13 session 1: patient 22 part 1


In [278]:
# subject 10 / patient 23 session 1 has a part 2
part_one_info = mapped_infos[10][1]
part_two_info = None
for info in infos:
    if info.Patient.values[0] == 23 and info.Part.values[0] == 2:
        part_two_info = info
        break
assert part_two_info is not None
# append part 2 to part 1
mapped_infos[10][1] = pd.concat([part_one_info, part_two_info])

In [295]:
for subject, sessions in mapped_infos.items():
    for session, info in sessions.items():
        should_times = response_times[subject][session]
        actual_times = info.ResponseTime.values

        if len(should_times) != len(actual_times):
            print(
                f"Subject {subject} session {session} has {len(should_times)} should times and {len(actual_times)} actual times")
            for i, should_time in enumerate(should_times):
                while not np.isclose(should_time, actual_times[i]):
                    # remove row
                    print(f"Removing row {i} from subject {subject} session {session}")
                    info.drop(i, inplace=True)
                    actual_times = info.ResponseTime.values
            assert len(should_times) == len(actual_times)


In [296]:
# Enumerate TrialNumber
for subject, sessions in mapped_infos.items():
    for session, info in sessions.items():
        info.TrialNumber = np.arange(1, len(info) + 1)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192]


In [297]:
# Write to csv
out_dir = join(_get_in_dir(), "to_convert", project, "micro_data")
for subject, sessions in mapped_infos.items():
    for session, info in sessions.items():
        uncorrected_patient = info.Patient.values[0]
        file = join(out_dir,
                    f"Micro_Intervals_Patient_{uncorrected_patient:02}_Session_{session:02}_Part_01_Interval_0_NaN_s.csv")
        info.to_csv(file, index=False)

# Visual Task (now for real)

In [251]:
subject = 10
session = 1
ctx = create_context(subject, session)

In [250]:
micro_files = _find_micro_data_files(ctx)
for subject, sessions in micro_files.items():
    for session, parts in sessions.items():
        for part, file in parts.items():
            print(f"Subject {subject} session {session} part {part}: {file}")

Subject 1 session 1 part 1: C:\Users\conta\git\janhohenheim\usz-neuro-conversion\in\to_convert\Human_MTL_units_visual_WM\micro_data\Micro_Intervals_Patient_13_Session_01_Part_01_Interval_0_NaN_s.mat
Subject 9 session 1 part 1: C:\Users\conta\git\janhohenheim\usz-neuro-conversion\in\to_convert\Human_MTL_units_visual_WM\micro_data\Micro_Intervals_Patient_19_Session_01_Part_01_Interval_0_NaN_s.mat
Subject 13 session 1 part 1: C:\Users\conta\git\janhohenheim\usz-neuro-conversion\in\to_convert\Human_MTL_units_visual_WM\micro_data\Micro_Intervals_Patient_22_Session_01_Part_01_Interval_0_NaN_s.mat
Subject 10 session 1 part 1: C:\Users\conta\git\janhohenheim\usz-neuro-conversion\in\to_convert\Human_MTL_units_visual_WM\micro_data\Micro_Intervals_Patient_23_Session_01_Part_01_Interval_0_NaN_s.mat
Subject 10 session 1 part 2: C:\Users\conta\git\janhohenheim\usz-neuro-conversion\in\to_convert\Human_MTL_units_visual_WM\micro_data\Micro_Intervals_Patient_23_Session_01_Part_02_Interval_0_NaN_s.mat
Su

In [258]:
info = None


def get_matlab_trial_info(ctx: SessionContext) -> pd.DataFrame:
    global info
    if info is not None:
        return info
    micro_files = _find_micro_data_files(ctx)
    files = micro_files[ctx.subject]
    reference_file = files[ctx.session][1]  # all csvs are called part 1
    csv_name = reference_file.replace(".mat", ".csv")
    with open(csv_name, "r") as file:
        info = pd.read_csv(file, sep=",")
    return info


def get_trial_indices(ctx: SessionContext) -> List[int]:
    micro_info = get_matlab_trial_info(ctx)
    # Source: https://stackoverflow.com/a/17215844
    x = micro_info.loc[:, "Session"] == ctx.session
    return x[x].index.values


@dataclass(frozen=True)
class MicroData:
    matrix: np.ndarray
    trial_info: pd.DataFrame
    timestamps_per_trial: np.ndarray


def prepare_micro_data(ctx: SessionContext) -> MicroData:
    micro_files = _find_micro_data_files(ctx)
    files = micro_files[ctx.subject][ctx.session]

    file = files[1]

    with h5py.File(file, 'r') as file:
        ref = file.get("data/trial")[0][0]

        matrix = np.array(file[ref][:])

        trial_info = get_matlab_trial_info(ctx)

        times_ref = file.get("data/time")[0][0]
        times = file[times_ref][:][:, 0]

        assert matrix.shape[0] == times.shape[0]
        assert abs(matrix[-1, -1]) > 1e-6
        assert times[-1] > 1

        if len(files) > 1:
            assert len(files) == 2
            with h5py.File(files[2], 'r') as file:
                ref = file.get("data/trial")[0][0]
                matrix = np.concatenate((matrix, np.array(file[ref][:])))
                times_ref = file.get("data/time")[0][0]

                last_stop_timestamp_of_part_one = trial_info.loc[(trial_info.Part == 1), "TimeStopTimestamp"].max()
                first_start_timestamp_of_part_two = trial_info.loc[(trial_info.Part == 2), "TimeStartTimestamp"].min()
                offset = first_start_timestamp_of_part_two - last_stop_timestamp_of_part_one
                offset = offset / 1e6  # mus to s

                second_part_times = file[times_ref][:][:, 0] + offset
                times = np.concatenate((times, second_part_times))
                assert matrix.shape[0] == times.shape[0]
                assert abs(matrix[-1, -1]) > 1e-6
                assert times[-1] > 1

        return MicroData(
            matrix=matrix,
            trial_info=trial_info,
            timestamps_per_trial=times
        )

In [259]:
info = get_matlab_trial_info(ctx)

In [260]:
micro_data = prepare_micro_data(ctx)

In [261]:
micro_data.timestamps_per_trial[-1]

814.96679925